In [1]:
import time, datetime
import locale

import pandas as pd

import requests
from bs4 import BeautifulSoup

## Задание 1.
### Обязательная часть

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.


In [2]:
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
KEYWORDS = {'python', 'парсинг', 'JavaScript'}

df = pd.DataFrame(columns=['Date', 'Title', 'Link'])
current_row = 0

# get rescent posts
res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html5lib')
posts = soup.find_all('article', class_='post')

for post in posts:
    # check if a post have id
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
        
    # find and check all preview information
    title = post.find('h2', class_='post__title').text.lower()
    kw_in_title = [kw in title for kw in KEYWORDS]
    
    preview_text = post.find('div', class_='post__text').text.lower()
    kw_in_preview = [kw in preview_text for kw in KEYWORDS]
    
    hubs = post.find_all('a', class_='hub-link')
    current_hubs = set()
    for hub in hubs:
        current_hubs.add(hub.text.lower())
        
    if current_hubs.intersection(KEYWORDS) or any(kw_in_title) or any(kw_in_preview):
        # parse link
        post_header = post.find('a', class_ = 'post__title_link')
        link = post_header.attrs.get('href')

        # obtain date-time
        datetime_block = post.find_all('span', class_='post__time')
        datetime_string = datetime_block[0].contents[0]
        try:
            dt = pd.to_datetime(datetime_string, format='%d %B %Y в %H:%M')
        except ValueError:
            if 'сегодня' in datetime_string:
                date = datetime.date.today()
            elif 'вчера' in datetime_string:
                date = datetime.date.today() - datetime.timedelta(days=1)
            time_str = datetime_string.split()[-1]
            dt = pd.to_datetime(str(date) + ' ' + time_str)
        # update dataframe
        df.loc[current_row] = dt, title, link, 
        current_row += 1
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

df

,Date,Title,Link
0,2020-09-01 09:40:00,\n как мы научили робота чувству юмора\n,https://habr.com/ru/company/funcorp/blog/517182/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>


In [3]:
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
KEYWORDS = {'python', 'парсинг', 'JavaScript'}

df = pd.DataFrame(columns=['Date', 'Title', 'Link', 'hubs'])
current_row = 0

res = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(res.text, 'html5lib')
posts = soup.find_all('article', class_='post')

for post in posts:
    post_id = post.parent.attrs.get('id')
    if not post_id:
        continue
        
        
    hubs = post.find_all('a', class_='hub-link')
    current_hubs = set()
    for hub in hubs:
        current_hubs.add(hub.contents[0].lower())
        
    post_header = post.find_all('a', class_ = 'post__title_link')
    link = post_header[0].attrs.get('href')
    title = post_header[0].contents[0]
    
    current_req = requests.get(link)
    current_soup = BeautifulSoup(current_req.text, 'html.parser')
    post_text = current_soup.find("div", {"class": "post__text"}).text # from https://habr.com/ru/post/346198/
    kw_in_text = [kw in post_text for kw in KEYWORDS]
    kw_in_title = [kw in title for kw in KEYWORDS]
    
    if current_hubs.intersection(KEYWORDS) or any(kw_in_text) or any(kw_in_title):
        post_header = post.find_all('a', class_ = 'post__title_link')
        link = post_header[0].attrs.get('href')
        title = post_header[0].contents[0]

        datetime_block = post.find_all('span', class_='post__time')
        datetime_string = datetime_block[0].contents[0]
        try:
            dt = pd.to_datetime(datetime_string, format='%d %B %Y в %H:%M')
        except ValueError:
            if 'сегодня' in datetime_string:
                date = datetime.date.today()
            elif 'вчера' in datetime_string:
                date = datetime.date.today() - datetime.timedelta(days=1)
            time_str = datetime_string.split()[-1]
            dt = pd.to_datetime(str(date) + ' ' + time_str)
        df.loc[current_row] = dt, title, link, current_hubs
        current_row += 1
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8')

df

,Date,Title,Link,hubs
0,2020-09-01 10:00:00,"Awesome-лист своими руками, или GitHub вместо ...",https://habr.com/ru/company/croc/blog/496594/,"{github, администрирование баз данных, блог ко..."
1,2020-09-01 09:40:00,Как мы научили робота чувству юмора,https://habr.com/ru/company/funcorp/blog/517182/,"{python, машинное обучение, блог компании func..."


## Задание 2.
### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Check. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.


In [4]:
EMAILS = ['xxx@x.ru', 'yyy@y.com']

res = requests.post('https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data',
                    headers = {'Vaar-Header-App-Product': 'hackcheck-web-avast', 'Vaar-Version': '0'},
                    json = {"emailAddresses":EMAILS})

In [5]:
import json
info = json.loads(res.text)

df = pd.DataFrame(columns=['email', 'breach_date', 'breach_source', 'breach_description'])
current_row = 0

for email in EMAILS:
    for breach_id in info['summary'][email]['breaches']:
        breach = info['breaches'][str(breach_id)]
        df.loc[current_row] = email, breach['publishDate'], breach['site'], breach['description']
        current_row += 1
df

,email,breach_date,breach_source,breach_description
0,xxx@x.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
1,xxx@x.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
2,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
3,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
4,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
5,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
6,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
7,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
8,yyy@y.com,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
9,yyy@y.com,2019-02-21T00:00:00Z,www.dangdang.com,"This is a list of email addresses only, and as..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [6]:
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'

VERSION = '5.103'
SLEEP = 0.33

In [7]:
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'group': 'netology',
    'owner_id':'-59793580',
    'count': 50
}
res = requests.get(NEWSFEED_REQUEST, params)
res

<Response [200]>

In [8]:
pd.DataFrame(
    [[pd.to_datetime(x['date'], unit='s'), x['text']] for x in res.json()['response']['items']],
    columns = ['date', 'text']
)

,date,text
0,2020-09-01 07:10:00,Гренландия 16+ \n С 3 сентября Кинотеатр Комсо...
1,2020-08-31 15:10:00,Вратарь Галактики 6+ \nС 3 сентября Кинотеатр ...
2,2020-08-31 10:10:00,Довод 18+\nС 3 сентября Кинотеатр Комсомолец\n...
3,2020-08-31 09:12:29,Поезд в Пусан 2: Полуостров 18+\nС 3 сентября ...
4,2020-08-28 07:13:51,"✅Дорогие друзья, присоединяйтесь к голосованию!!!"
5,2020-08-27 11:17:00,Тролли. Мировой тур 2D 6+ \nСовсем скоро!!! Ки...
6,2020-08-27 11:03:56,27 АВГУСТА ДЕНЬ РОССИЙСКОГО КИНО!!! 😁\nВСЕХ С ...
7,2020-08-27 09:08:09,
8,2020-08-07 12:35:02,"В начале месяца, 1 августа 2020, как сообщалос..."
9,2020-07-01 14:58:39,
